In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

model_name = "openai:gpt-4o-mini"

In [21]:
system_prompt = """
You are tasked with processing a list of messages. Each message contains:

Your goal is to:
1. **Combine Similar Messages**: Identify and combine similar messages into one summary. 
   - Similar messages should be grouped together under a single title.
   - Keep all details intact—do not add or remove information.

2. **Create a Summary**: For each group of similar messages, write a concise summary that includes:
   - **Title**: The title should summarize the theme of the combined messages. It should be clear, concise, and relevant to the content.
   - **Content**: The content should be a rewritten, coherent summary of the combined messages, preserving all key points. Ensure the text flows naturally.
   - **Metadata**: Merge metadata from the similar messages into one dictionary. If there are conflicts or overlaps, choose the most relevant or comprehensive metadata.

3. **Length Limit**: The **title** and **content** together should not exceed **1024 characters** in total.
   - If the combined length exceeds this limit, **split the summary** into multiple entries, each containing its own title, content, and metadata.

### **Instructions**:
- Identify messages that are similar, based on the content or topic.
- When combining them:
  - Keep the content intact.
  - Do not add or infer new information.
  - Only merge messages that share the same theme or subject.

### **Output Format**:
Your response should be a list of summaries, where each summary is represented as a dictionary with the following fields:
```json
[
  {
    "title": "Title of the Combined Summary",
    "content": "The full, rewritten text of the combined summary",
    "metadata": { 
      "key1": "value1", 
      "key2": "value2" 
    }
  },
  {
    "title": "Title of the Another Combined Summary",
    "content": "The full, rewritten text of the another combined summary",
    "metadata": {
      "key1": "value1", 
      "key2": "value2" 
    }
  }
]

DOUBLE CHECK YOUR ANSWER!!!
"""

In [22]:
from pydantic import BaseModel, Field


class SummaryDto(BaseModel):
    """Represents a user with personal details such as id, username, first name, last name, email, password, phone, and user status."""

    title: str = Field(
        ..., description="News title", example="Rosneft announced dividend payment"
    )
    content: str = Field(
        ...,
        description="News text",
        example="A robotic dog with a rabbit and a minigun is a new word in home security. A Chinese enthusiast has created an unusual hybrid, where a cute animal has become part of the combat system. Now this mecha-hare not only looks threatening, but also effectively performs its task, monitoring the yard.",
    )
    metadata: dict = Field(
        ...,
        description="Metadata",
        example={"source": "https://www.google.com"},
    )

In [23]:
from typing import List
from pydantic_ai import Agent


agent = Agent(
    model=model_name,
    deps_type=List[str],
    output_type=List[SummaryDto],
    system_prompt=system_prompt,
    verbose=True
)

In [24]:
from typing import Any, Dict


class Message(BaseModel):
    content: str = Field(..., description="The text content of the message")
    metadata: Dict[str, Any] = Field(
        default_factory=dict,
        description="Additional metadata about the message (e.g., timestamp, author, etc.)",
    )

In [25]:
messages: List[Message] = []


In [26]:
messages = [
    Message(
        content="Apple launches the new iPhone 15 with advanced features including better cameras, a faster processor, and longer battery life. The iPhone 15 is expected to set new records for sales worldwide.",
        metadata={"timestamp": "2025-04-15T08:00:00Z", "author": "Tech News"}
    ),
    Message(
        content="Samsung's Galaxy S23 series has been well received, with improvements in screen quality, battery efficiency, and camera performance. Analysts believe the Galaxy S23 will compete fiercely with Apple's new iPhone 15 in the coming months.",
        metadata={"timestamp": "2025-04-15T09:00:00Z", "author": "Gadget Review"}
    ),
    Message(
        content="The new iPhone 15 has been announced by Apple, showcasing features like improved battery life, upgraded cameras, and a faster chip. This new release has generated a lot of buzz in the tech community.",
        metadata={"timestamp": "2025-04-15T08:30:00Z", "author": "Tech Buzz"}
    ),
    Message(
        content="Microsoft has released a major update to Windows 11, with a focus on security improvements and performance enhancements. The update is designed to make the user experience smoother and more secure, especially for business environments.",
        metadata={"timestamp": "2025-04-15T10:00:00Z", "author": "Tech Review"}
    ),
    Message(
        content="Apple announces the iPhone 15 with new features. The improved camera and faster performance are the highlights, and it’s expected to be a major success in the smartphone market, surpassing previous iPhone models in sales.",
        metadata={"timestamp": "2025-04-15T09:30:00Z", "author": "Gadget Watch"}
    ),
    Message(
        content="Sony PlayStation 5 continues to dominate the gaming market, with new exclusive games driving console sales. The PlayStation 5 is set to remain a strong competitor in the gaming industry for the foreseeable future.",
        metadata={"timestamp": "2025-04-15T11:00:00Z", "author": "Gaming Weekly"}
    ),
    Message(
        content="Apple's new iPhone 15 features include an improved camera system, a larger battery, and faster processing speeds. These features are expected to appeal to both new buyers and loyal Apple fans.",
        metadata={"timestamp": "2025-04-15T07:45:00Z", "author": "Tech World"}
    ),
    Message(
        content="Microsoft's latest move in cloud computing involves the expansion of Azure services into new international markets. This strategic move is designed to compete more effectively with Amazon Web Services and Google Cloud.",
        metadata={"timestamp": "2025-04-15T12:00:00Z", "author": "Cloud Tech"}
    ),
    Message(
        content="Apple introduces a more powerful chip in the iPhone 15, offering significant performance gains. With an upgraded display and better battery optimization, this new iPhone is likely to appeal to tech enthusiasts and professionals alike.",
        metadata={"timestamp": "2025-04-15T08:45:00Z", "author": "Tech Insights"}
    ),
    Message(
        content="Microsoft is pushing for greater security features in Windows 11. The latest update includes critical patches and enhancements to prevent cyber attacks, making it a safer platform for both personal and enterprise users.",
        metadata={"timestamp": "2025-04-15T10:30:00Z", "author": "Security Daily"}
    ),
]

In [27]:
import json


messages_json = [json.dumps(message.dict()) for message in messages]
result = await agent.run(messages_json)

C:\Users\abaghdasaryan\AppData\Local\Temp\ipykernel_19856\2566462569.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  messages_json = [json.dumps(message.dict()) for message in messages]


In [28]:
result.output

[SummaryDto(title='Apple Launches iPhone 15 With Advanced Features', content='Apple launches the new iPhone 15 with advanced features including improved camera systems, a faster processor, and longer battery life. This new release has generated a lot of buzz in the tech community, and it’s expected to set new records for sales worldwide, surpassing previous iPhone models. The iPhone 15 aims to appeal to both new buyers and loyal Apple fans.', metadata={'timestamp': '2025-04-15T08:00:00Z', 'author': 'Tech News'}),
 SummaryDto(title='Samsung Galaxy S23 Competes with iPhone 15', content="Samsung's Galaxy S23 series has been well received, boasting improvements in screen quality, battery efficiency, and camera performance. Analysts believe it will compete fiercely with Apple's new iPhone 15 in the coming months.", metadata={'timestamp': '2025-04-15T09:00:00Z', 'author': 'Gadget Review'}),
 SummaryDto(title='Microsoft Introduces Major Windows 11 Update', content="Microsoft has released a ma

In [30]:
for summary in result.output:
    print(f"""
TITLE: {summary.title}
CONTENT: {summary.content}
METADATA: {summary.metadata}
""")


TITLE: Apple Launches iPhone 15 With Advanced Features
CONTENT: Apple launches the new iPhone 15 with advanced features including improved camera systems, a faster processor, and longer battery life. This new release has generated a lot of buzz in the tech community, and it’s expected to set new records for sales worldwide, surpassing previous iPhone models. The iPhone 15 aims to appeal to both new buyers and loyal Apple fans.
METADATA: {'timestamp': '2025-04-15T08:00:00Z', 'author': 'Tech News'}


TITLE: Samsung Galaxy S23 Competes with iPhone 15
CONTENT: Samsung's Galaxy S23 series has been well received, boasting improvements in screen quality, battery efficiency, and camera performance. Analysts believe it will compete fiercely with Apple's new iPhone 15 in the coming months.
METADATA: {'timestamp': '2025-04-15T09:00:00Z', 'author': 'Gadget Review'}


TITLE: Microsoft Introduces Major Windows 11 Update
CONTENT: Microsoft has released a major update to Windows 11, focusing on secur